In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time


from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options


from selenium.common.exceptions import TimeoutException, NoSuchElementException



from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time



In [7]:
def quit_existing_drivers():
      try:
            driver = webdriver.Firefox(service=service, options=firefox_options)
            driver.quit()
      except:
            pass
      


def get_table_data(driver,firsttime):
    try:
        
        # first_button = WebDriverWait(driver, 30).until(
        #     EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/header/div[1]/div/div[2]/div/div/div/div/div/nav/div/ul/li[5]/a"))
        # )
        # first_button.click()
        
        # Wait for the button to be clickable and click it
        if(firsttime):
                  button = WebDriverWait(driver, 20).until(
                      EC.element_to_be_clickable((By.XPATH,"/html/body/table/tbody/tr[4]/td/button"))
                      )
                  button.click()

        # Wait for the table to load after clicking the button
        table = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )
        
        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Try to find the table
        table = soup.find('table')
        
        if table is None:
            print("Table not found. Printing page source:")
            print(soup.prettify())
            return []
        
        data = []
        # Extract table rows
        for row in table.find_all('tr')[1:]:  # Skip the header row
            row_data = [cell.text.strip() for cell in row.find_all('td')]
            data.append(row_data)
        
        return data
    except TimeoutException:
        print("Timeout waiting for button or table to load. Printing page source:")
        print(BeautifulSoup(driver.page_source, 'html.parser').prettify())
        return []
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []
    
    

# Function to get the selected activity from the peptide page
def get_selected_activity(driver, zm_id):
    peptide_url = f'https://biochemia.uwm.edu.pl/biopep/peptide_data_page1.php?zm_ID={zm_id}'
    driver.get(peptide_url)
    
    try:
        # Wait until the select element is present
        select_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "activity"))
        )

        # Get the selected option (activity)
        selected_option = select_element.find_element(By.CSS_SELECTOR, "option:checked")
        selected_activity = selected_option.text
        
        # Find the input element with the name 'txt_inchikey' and get its value
        input_element = driver.find_element(By.NAME, "txt_inchikey")
        inchikey_value = input_element.get_attribute('value')
        
        return selected_activity, inchikey_value
    except Exception as e:
        print(f"Error while getting selected activity for zm_ID {zm_id}: {str(e)}")
        return None


In [9]:

# Replace this path with the actual path where you saved geckodriver.exe
geckodriver_path = r"C:\Users\ASUS\geckodriver-v0.34.0-win64\geckodriver.exe"
service = Service(geckodriver_path)

firefox_options = Options()
# Uncomment the next line if you want to run Firefox in headless mode
# firefox_options.add_argument("--headless")

# Rest of your functions (get_table_data, etc.) go here...

# URL of the page containing the table
# url = 'https://biochemia.uwm.edu.pl/biopep-uwm/'
url = 'https://biochemia.uwm.edu.pl/biopep/start_biopep.php'

# Quit any existing drivers
quit_existing_drivers()

# Initialize the webdriver
driver = None
try:
    driver = webdriver.Firefox(service=service, options=firefox_options)
    driver.get(url)

    all_data = []
    page = 1

    firsttime = True
    while (page<=337):
        print(f"Scraping page {page}...")
        
        # Get data from the current page
        page_data = get_table_data(driver,firsttime)
        firsttime = False
        if not page_data:
            print(f"No data found on page {page}. Stopping.")
            break
        
        # Visit the peptide page for each entry and replace [0] with the selected activity
        for entry in page_data:
            zm_id = entry[1]  # Assuming the ID is at position [1] in the entry
            activity, inchikey_value = get_selected_activity(driver, zm_id)
            if activity and inchikey_value:
                entry[0] = activity  # Replace [0] with the selected activity
                entry.append(inchikey_value)  # Add the InChIKey value as a new column
            driver.back()
        
        all_data.extend(page_data)
        
        # Find the "Next" button
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[text()='Next']"))
            )
            next_button.click()
            page += 1
            
            # Add a delay to allow the page to load
            time.sleep(3)
        except:
            print("Reached the last page or couldn't find 'Next' button")
            break

    # Get headers from the first page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    # if table:
    #     headers = [header.text.strip() for header in table.find_all('th')]
    # else:
    #     print("Couldn't find table for headers. Using default headers.")
    #     headers = [f"Column_{i}" for i in range(len(all_data[0])) if all_data]

    headers = all_data[0] if all_data else []
    
    # Create a DataFrame
    if all_data:
        df = pd.DataFrame(all_data[1:], columns=headers)
        # Save to CSV
        df.to_csv('bioactive_peptides_all_pages.csv', index=False)
        print(f"Data from {page} pages has been scraped and saved to 'bioactive_peptides_all_pages.csv'")
    else:
        print("No data was scraped. Please check the website structure and your internet connection.")

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    # Close the browser
    if driver:
        driver.quit()

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

In [ ]:

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

def extract_info(text):
    """Extracts SMILES, InChI, and InChIKey from the given text."""

    # Regular expressions to match the desired patterns
    smiles_pattern = r"SMILES: (.+)"
    inchi_pattern = r"InChI=(.+)"
    inchikey_pattern = r"InChIKey: (.+)"

    # Create an empty dictionary to store the extracted information
    info = {}

    # Search for matches in the text
    smiles_match = re.search(smiles_pattern, text)
    inchi_match = re.search(inchi_pattern, text)
    inchikey_match = re.search(inchikey_pattern, text)

    # Extract the values if matches are found
    if smiles_match:
        info['SMILES'] = smiles_match.group(1)
    if inchi_match:
        info['InChI'] = inchi_match.group(1)
    if inchikey_match:
        info['InChIKey'] = inchikey_match.group(1)

    return info


# Function to get the selected activity from the peptide page
def get_additional_info(driver, zm_id):
    peptide_url = f'https://biochemia.uwm.edu.pl/biopep/peptidedatacard.php?zm_ID={zm_id}'
    driver.get(peptide_url)
    
    try:
        # Extract full page text content
        text_content = driver.find_element(By.TAG_NAME, "body").text
        
        # Extract SMILES, InChI, and InChIKey using the extract_info function
        chemical_info = extract_info(text_content)
        
        return chemical_info
    
    except Exception as e:
        print(f"Error while getting additional info for zm_ID {zm_id}: {str(e)}")
        return None


In [23]:
# Replace this path with the actual path where you saved geckodriver.exe
geckodriver_path = r"C:\Users\ASUS\geckodriver-v0.34.0-win64\geckodriver.exe"
service = Service(geckodriver_path)

firefox_options = Options()
# Uncomment the next line if you want to run Firefox in headless mode
# firefox_options.add_argument("--headless")

# Initialize the webdriver
driver = webdriver.Firefox(service=service, options=firefox_options)

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    zm_id = row["ID"]  # Assuming the ID column is named "ID"
    print(f"Scraping for zm_ID: {zm_id}")

    # Call get_additional_info and handle potential errors
    result = get_additional_info(driver, zm_id)
    # print(result)
    if result:
        df.at[index, "SMILES"] = result.get("SMILES", None)
        df.at[index, "InChI"] = result.get("InChI", None)
        df.at[index, "InChIKey"] = result.get("InChIKey", None)
    else:
        print(f"Error scraping data for zm_ID: {zm_id}")
    print(f"  Progress: {100 * (index + 1) / len(df):.2f}%")  # Track progress
    

# Quit the webdriver
driver.quit()

# Save the updated DataFrame (optional)
# df.to_csv("data_with_additional_info.csv", index=False)

In [25]:
df.drop(['inchikey'], axis=1, inplace=True)
df.to_csv('bioactive_peptides_all_pages.csv', index=False)

In [104]:
##############################
import pandas as pd


In [105]:
df = pd.read_csv('./bioactive_peptides_all_pages.csv')
df.drop_duplicates()
print(df.shape)
print(df.columns)

(5383, 10)
Index(['activity', 'ID', 'Name', 'Sequence', 'Chem. mass', 'Monois. mass',
       'Unnamed: 6', 'SMILES', 'InChI', 'InChIKey'],
      dtype='object')


In [106]:
df.head()

,activity,ID,Name,Sequence,Chem. mass,Monois. mass,Unnamed: 6,SMILES,InChI,InChIKey
0,regulating | re,2566,regulating cell-permeability peptide,NYKKPKL,890.0780,889.5369,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C42H71N11O10/c1-25(2)22-33(42(62)63)52-38(5...,RQFNGWRPSKHMKK-NXBWRCJVSA-N
1,regulating | re,2567,regulating cell-permeability peptide,NYKKPKLAAAPALLALLVAPLLAVAA,2601.2117,2599.6143,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-108(166)84...,SLFAVMFXXFBIEI-PHOGLEJUSA-N
2,regulating | re,2568,regulating cell-permeability peptide,AAVALLPAVLLALLAPAAANYKKPKL,2601.2117,2599.6143,0.00 EC50,N[C@@]([H])(C)C(=O)N[C@@]([H])(C)C(=O)N[C@@]([...,1S/C125H214N30O29/c1-62(2)53-85(145-113(171)89...,SCGYKWYBPYHFEB-PHOGLEJUSA-N
3,regulating | re,2569,regulating cell-permeability peptide,NYKKPKLAAAAAVALLPAVLLALLAP,2601.2117,2599.6143,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-110(168)84...,FCRVOEOQSNIOPS-PHOGLEJUSA-N
4,opioid | op,2570,VV-hemorphin-7,VVYPWTQRF,1195.3651,1194.6167,0.00 EC50,NaN,1S/C59H82N14O13/c1-31(2)47(61)54(81)71-48(32(3...,FFBBHLKDMHCFTH-AKYHLAPZSA-N


In [107]:
# print(df['Unnamed: 6'].isna().sum())
df = df.dropna()
df.shape

(1019, 10)

In [108]:
df.head()

,activity,ID,Name,Sequence,Chem. mass,Monois. mass,Unnamed: 6,SMILES,InChI,InChIKey
0,regulating | re,2566,regulating cell-permeability peptide,NYKKPKL,890.0780,889.5369,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C42H71N11O10/c1-25(2)22-33(42(62)63)52-38(5...,RQFNGWRPSKHMKK-NXBWRCJVSA-N
1,regulating | re,2567,regulating cell-permeability peptide,NYKKPKLAAAPALLALLVAPLLAVAA,2601.2117,2599.6143,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-108(166)84...,SLFAVMFXXFBIEI-PHOGLEJUSA-N
2,regulating | re,2568,regulating cell-permeability peptide,AAVALLPAVLLALLAPAAANYKKPKL,2601.2117,2599.6143,0.00 EC50,N[C@@]([H])(C)C(=O)N[C@@]([H])(C)C(=O)N[C@@]([...,1S/C125H214N30O29/c1-62(2)53-85(145-113(171)89...,SCGYKWYBPYHFEB-PHOGLEJUSA-N
3,regulating | re,2569,regulating cell-permeability peptide,NYKKPKLAAAAAVALLPAVLLALLAP,2601.2117,2599.6143,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-110(168)84...,FCRVOEOQSNIOPS-PHOGLEJUSA-N
5,opioid | op,2571,VV-hemorphin-5,VVYPWTQ,892.0064,891.4476,0.00 EC50,N[C@@H](C(C)C)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](...,1S/C44H61N9O11/c1-22(2)35(46)40(59)51-36(23(3)...,VAEOIFAHJWMTKD-NMUVPRMFSA-N


In [109]:
### keeping - id, sequence, inchi key, ic50/ec50, activity code, name, activity
df.drop(['Chem. mass','Monois. mass'], axis=1, inplace=True)
print(df.shape)

df['activity_code'] = df['activity'].apply(lambda x: x.split('|')[-1])
df['activity'] = df['activity'].apply(lambda x: x.split('|')[0])


(1019, 8)


In [110]:
df.head()

,activity,ID,Name,Sequence,Unnamed: 6,SMILES,InChI,InChIKey,activity_code
0,regulating,2566,regulating cell-permeability peptide,NYKKPKL,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C42H71N11O10/c1-25(2)22-33(42(62)63)52-38(5...,RQFNGWRPSKHMKK-NXBWRCJVSA-N,re
1,regulating,2567,regulating cell-permeability peptide,NYKKPKLAAAPALLALLVAPLLAVAA,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-108(166)84...,SLFAVMFXXFBIEI-PHOGLEJUSA-N,re
2,regulating,2568,regulating cell-permeability peptide,AAVALLPAVLLALLAPAAANYKKPKL,0.00 EC50,N[C@@]([H])(C)C(=O)N[C@@]([H])(C)C(=O)N[C@@]([...,1S/C125H214N30O29/c1-62(2)53-85(145-113(171)89...,SCGYKWYBPYHFEB-PHOGLEJUSA-N,re
3,regulating,2569,regulating cell-permeability peptide,NYKKPKLAAAAAVALLPAVLLALLAP,0.00 EC50,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-110(168)84...,FCRVOEOQSNIOPS-PHOGLEJUSA-N,re
5,opioid,2571,VV-hemorphin-5,VVYPWTQ,0.00 EC50,N[C@@H](C(C)C)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](...,1S/C44H61N9O11/c1-22(2)35(46)40(59)51-36(23(3)...,VAEOIFAHJWMTKD-NMUVPRMFSA-N,op


In [111]:
def convert_value(value):
      """Converts a string value like "0.00 EC50" or "0.00 IC50" to a float, negating if it contains "IC50"."""
      try:
            # Extract the numeric value
            split_value = value.split()
            numeric_value = float(split_value[0])
          
            # Negate if the value contains "IC50"
            if "IC50" in value:
                numeric_value = -numeric_value
            
            return numeric_value
      except ValueError:
            # Handle cases where the value cannot be converted to a float
            return None
      
df['Unnamed: 6'] = df['Unnamed: 6'].apply(convert_value)

In [112]:
df.head()

,activity,ID,Name,Sequence,Unnamed: 6,SMILES,InChI,InChIKey,activity_code
0,regulating,2566,regulating cell-permeability peptide,NYKKPKL,0.0,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C42H71N11O10/c1-25(2)22-33(42(62)63)52-38(5...,RQFNGWRPSKHMKK-NXBWRCJVSA-N,re
1,regulating,2567,regulating cell-permeability peptide,NYKKPKLAAAPALLALLVAPLLAVAA,0.0,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-108(166)84...,SLFAVMFXXFBIEI-PHOGLEJUSA-N,re
2,regulating,2568,regulating cell-permeability peptide,AAVALLPAVLLALLAPAAANYKKPKL,0.0,N[C@@]([H])(C)C(=O)N[C@@]([H])(C)C(=O)N[C@@]([...,1S/C125H214N30O29/c1-62(2)53-85(145-113(171)89...,SCGYKWYBPYHFEB-PHOGLEJUSA-N,re
3,regulating,2569,regulating cell-permeability peptide,NYKKPKLAAAAAVALLPAVLLALLAP,0.0,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,1S/C125H214N30O29/c1-62(2)53-86(146-110(168)84...,FCRVOEOQSNIOPS-PHOGLEJUSA-N,re
5,opioid,2571,VV-hemorphin-5,VVYPWTQ,0.0,N[C@@H](C(C)C)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](...,1S/C44H61N9O11/c1-22(2)35(46)40(59)51-36(23(3)...,VAEOIFAHJWMTKD-NMUVPRMFSA-N,op


In [113]:
df = df.reindex(columns=['ID', 'Sequence', 'InChI','SMILES','InChIKey', 'Unnamed: 6', 'activity_code', 'Name', 'activity'])
df = df.rename(columns={'Unnamed: 6': 'EC50'})
print(df.columns)
df.head()

Index(['ID', 'Sequence', 'InChI', 'SMILES', 'InChIKey', 'EC50',
       'activity_code', 'Name', 'activity'],
      dtype='object')


,ID,Sequence,InChI,SMILES,InChIKey,EC50,activity_code,Name,activity
0,2566,NYKKPKL,1S/C42H71N11O10/c1-25(2)22-33(42(62)63)52-38(5...,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,RQFNGWRPSKHMKK-NXBWRCJVSA-N,0.0,re,regulating cell-permeability peptide,regulating
1,2567,NYKKPKLAAAPALLALLVAPLLAVAA,1S/C125H214N30O29/c1-62(2)53-86(146-108(166)84...,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,SLFAVMFXXFBIEI-PHOGLEJUSA-N,0.0,re,regulating cell-permeability peptide,regulating
2,2568,AAVALLPAVLLALLAPAAANYKKPKL,1S/C125H214N30O29/c1-62(2)53-85(145-113(171)89...,N[C@@]([H])(C)C(=O)N[C@@]([H])(C)C(=O)N[C@@]([...,SCGYKWYBPYHFEB-PHOGLEJUSA-N,0.0,re,regulating cell-permeability peptide,regulating
3,2569,NYKKPKLAAAAAVALLPAVLLALLAP,1S/C125H214N30O29/c1-62(2)53-86(146-110(168)84...,N[C@@]([H])(CC(=O)N)C(=O)N[C@@]([H])(Cc1ccc(O)...,FCRVOEOQSNIOPS-PHOGLEJUSA-N,0.0,re,regulating cell-permeability peptide,regulating
5,2571,VVYPWTQ,1S/C44H61N9O11/c1-22(2)35(46)40(59)51-36(23(3)...,N[C@@H](C(C)C)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](...,VAEOIFAHJWMTKD-NMUVPRMFSA-N,0.0,op,VV-hemorphin-5,opioid


In [114]:
df.shape

(1019, 9)

In [115]:
df.to_csv('bioactive_peptides_db.csv', index=False) #cleaned data

In [116]:
#######################

In [117]:
df2  = pd.read_csv('bioactive_peptides_db.csv')
df2 = df2.drop_duplicates()
df2.shape

(1019, 9)

In [118]:
df3 = df2[~df2['Sequence'].astype(str).apply(lambda x: x.startswith('<'))]
df3.shape

(1019, 9)

In [121]:
print("ID = ",len(df3['ID'].unique()))
print("Sequence = ",len(df3['Sequence'].unique()))
print("inchikey = ",len(df3['InChIKey'].unique()))
print("EC50 = ",len(df3['EC50'].unique()))
print("activity_code = ",len(df3['activity_code'].unique()))
print("Name = ",len(df3['Name'].unique()))
print("activity = ",len(df3['activity'].unique()))


ID =  1019
Sequence =  757
inchikey =  752
EC50 =  430
activity_code =  51
Name =  179
activity =  51


In [122]:
print(df3['activity_code'].unique())


[' re' ' op' ' ne' ' im' ' ctox' ' ab' ' am' ' ah' ' ac' ' sti' ' inh'
 ' ren' ' he' ' op1' ' is' ' op2' ' dpp' ' at' ' che' ' aami' ' apr'
 'Peptide \r\n      Data' ' ao' ' 35pd' ' hyp' ' glui' ' tox' ' HMGi'
 ' af' ' bin' ' dpp3' ' hypl' ' avi' ' hypc' ' ai' ' tpp2' ' cal1' ' nep2'
 ' neul' ' pseud' ' catb' ' xaap' ' pam' ' lcp' ' ttl' ' gluc' ' gluc2'
 ' caa' ' hur' ' aac2' ' nep']


In [123]:
df3 = df3[df3['activity_code']!='Peptide \r\n      Data']
df3.shape

(960, 9)

In [125]:
print("ID = ",len(df3['ID'].unique()))
print("Sequence = ",len(df3['Sequence'].unique()))
print("inchikey = ",len(df3['InChIKey'].unique()))
print("EC50 = ",len(df3['EC50'].unique()))
print("activity_code = ",len(df3['activity_code'].unique()))
print("Name = ",len(df3['Name'].unique()))
print("activity = ",len(df3['activity'].unique()))


ID =  960
Sequence =  716
inchikey =  711
EC50 =  413
activity_code =  50
Name =  169
activity =  50


In [126]:
df3.to_csv('biopep_db.csv', index=False) # to be used for training

In [127]:
df = pd.read_csv('bioactive_peptides_all_pages.csv')
df = df.drop_duplicates()
df.shape

(5048, 10)

In [128]:
df = df[~df['Sequence'].astype(str).apply(lambda x: x.startswith('<'))]
df.shape

(4994, 10)

In [129]:
ids = df3['ID'].unique()
df = df[~df['ID'].isin(ids)]
print(df.shape)

(4994, 10)


In [130]:
df= df['Sequence']
print(df)

0                          NYKKPKL
1       NYKKPKLAAAPALLALLVAPLLAVAA
2       AAVALLPAVLLALLAPAAANYKKPKL
3       NYKKPKLAAAAAVALLPAVLLALLAP
4                        VVYPWTQRF
                   ...            
5378                          GLFF
5379                          GLFF
5380                           LGF
5381                           LGF
5382                           LGF
Name: Sequence, Length: 4994, dtype: object


In [181]:
# df.to_csv('incomplete_db.csv', index=False) # to be used for testing

In [68]:
##############################################

In [182]:
## complete/raw = bioactive_peptides_all_pages.csv

## testing = incomplete.csv
## training = bioactive_peptides_db.csv


In [1]:
biopeptide_dict = {
    "aami": "alpha-amylase inhibitor",
    "am": "antiamnestic",
    "ah": "ACE inhibitor",
    "ctox": "celiac toxic",
    "ab": "antibacterial",
    "at": "antithrombotic",
    "im": "immunomodulating",
    "op": "opioid",
    "sti": "stimulating",
    "is": "immunostimulating",
    "ne": "neuropeptide",
    "re": "regulating",
    "ac": "anticancer",
    "ao": "antioxidative",
    "con": "contracting",
    "lig": "bacterial permease ligand",
    "ai": "anti-inflammatory",
    "nat": "natriuretic",
    "inh": "inhibitor",
    "che": "chemotactic",
    "acc": "accelerating",
    "he": "haemolytic",
    "an": "anorectic",
    "op1": "opioid agonist",
    "op2": "opioid antagonist",
    "rea": "reacting",
    "bin": "binding",
    "hyp": "hypotensive",
    "emb": "embryotoxic",
    "af": "antifungal",
    "avi": "antiviral",
    "vsc": "vasoconstrictor",
    "orp": "orphan receptor GPR14 agonist",
    "apr": "activating ubiquitin-mediated proteolysis",
    "bi": "bitter taste peptide",
    "um": "umami",
    "PKC": "Protein Kinase C inhibitor",
    "mac": "membrane-active peptide",
    "hep_bin": "heparin binding",
    "dpp": "dipeptidyl peptidase IV inhibitor",
    "tox": "toxic",
    "HMGi": "HMG-CoA reductase inhibitor",
    "glui": "alpha-glucosidase inhibitor",
    "cmk2": "CaMKII Inhibitor",
    "dpp3": "dipeptidyl peptidase III inhibitor",
    "35pd": "CaMPDE inhibitor",
    "ren": "renin inhibitor",
    "hypl": "hypolipidemic",
    "hypc": "hypocholesterolemic",
    "adb": "antidiabetic",
    "cal2": "Calpain 2 inhibitor",
    "lox": "lipoxygenase inhibitor",
    "cox1": "cyclooxygenase-1 inhibitor",
    "cox2": "cyclooxygenase-2 inhibitor",
    "plip": "pancreatic lipase inhibitor",
    "ace2": "ACE2 inhibitor",
    "bis": "bitterness suppressing",
    "ache": "AChE inhibitor",
    "bche": "BChE inhibitor",
    "tyr": "tyrosinase inhibitor",
    "ost": "osteoanabolic",
    "aapt": "anti-apoptotic",
    "ppar": "PPARγ antagonist",
    "far": "farnesyltransferase inhibitor",
    "cal1": "calpain 1 inhibitor",
    "catb": "cathepsin B inhibitor",
    "fur": "furin inhibitor",
    "xiap": "XIAP inhibitor",
    "pel": "pancreatic elastase inhibitor",
    "leut": "Leucyltransferase inhibitor",
    "pam": "PAM inhibitor",
    "xaap": "xaa-pro inhibitor",
    "lcp": "lactocepin inhibitor",
    "tpp2": "inhibitor of tripeptidyl peptidase II",
    "nep2": "neprilysin 2 inhibitor",
    "pla2": "phospholipase A2 inhibitor",
    "neul": "neurolysin inhibitor",
    "pseud": "pseudolysin inhibitor",
    "alph": "alkaline phosphatase inhibitor",
    "nep": "neprilysin inhibitor",
    "acylp": "acylaminoacyl peptidase inhibitor",
    "ttl": "tubulin-tyrosine ligase inhibitor",
    "acar": "alanine carboxypeptidase inhibitor",
    "gluc": "glutamate carboxypeptidase inhibitor",
    "gluc2": "glutamate carboxypeptidase II inhibitor",
    "caa": "inhibitor of cytosol alanyl aminopeptidase",
    "adha": "alcohol dehydrogenase activator",
    "hur": "hypouricemic",
    "dada": "D-Ala-D-Ala dipeptidase inhibitor",
    "aac2": "ACE 2 activator"
}

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('./bioactive_peptides_all_pages.csv')

print("original = ",df.shape)

df = df.drop_duplicates()

print("after dropping duplicates = ",df.shape)

original =  (5383, 10)
after dropping duplicates =  (5048, 10)


In [8]:
df.to_csv('bioactive_peptides_all_pages.csv', index=False)

In [19]:
count = df[["InChI", "SMILES"]].isnull().all(axis=1).sum()
# count = df[["InChI", "SMILES"]].isnull().any(axis=1).sum()
count

1416